In [ ]:
# https://bioconductor.org/packages/devel/bioc/vignettes/topGO/inst/doc/topGO.pdf
# https://sbc.shef.ac.uk/prostate-bioinformatics/rna-seq-gene-set-testing.nb.html

# https://bioconductor.org/packages/release/bioc/vignettes/fgsea/inst/doc/fgsea-tutorial.html
# https://ruins880.tistory.com/109
# https://www.gsea-msigdb.org/gsea/msigdb/collections.jsp
# https://www.gsea-msigdb.org/gsea/msigdb/human/collections.jsp#C5

# https://www.bioconductor.org/packages/release/bioc/manuals/GOfuncR/man/GOfuncR.pdf
# https://rpubs.com/Akhil_Velluva/GOfuncR

In [1]:
# if (!requireNamespace("BiocManager", quietly=TRUE))
# + install.packages("BiocManager")
# BiocManager::install("GOfuncR")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.2 (2023-10-31)

Installing package(s) 'GOfuncR'

Installation paths not writeable, unable to update packages
  path: /usr/local/lib/R/site-library
  packages:
    ps, roxygen2, RSQLite, withr

Old packages: 'affy', 'affyio', 'annotate', 'AnnotationFilter', 'beachmat',
  'BiocIO', 'biomaRt', 'biovizBase', 'BSgenome', 'bsseq', 'bumphunter',
  'ChAMP', 'ChAMPdata', 'ChromHeatMap', 'clusterProfiler', 'ComplexHeatmap',
  'cowplot', 'DelayedArray', 'DelayedMatrixStats', 'DMRcate', 'DNAcopy',
  'DOSE', 'DSS', 'edgeR', 'enrichplot', 'ensembldb', 'fgsea', 'genefilter',
  'geneLenDataBase', 'GenomicAlignments', 'GenomicFeatures', 'GEOquery',
  'ggfun', 'ggtree', 'globaltest', 'GO.db', 'GOSemSim', 'goseq', 'graph',
  'graphlayouts', 'HDF5Array', 'Ill

In [3]:
# if (!requireNamespace('BiocManager', quietly = TRUE))
# 	install.packages('BiocManager')
# BiocManager::install('Homo.sapiens')

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org



Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.2 (2023-10-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'Homo.sapiens'”
Installation paths not writeable, unable to update packages
  path: /usr/local/lib/R/site-library
  packages:
    ps, roxygen2, RSQLite, withr

Old packages: 'affy', 'affyio', 'annotate', 'AnnotationFilter', 'beachmat',
  'BiocIO', 'biomaRt', 'biovizBase', 'BSgenome', 'bsseq', 'bumphunter',
  'ChAMP', 'ChAMPdata', 'ChromHeatMap', 'clusterProfiler', 'ComplexHeatmap',
  'cowplot', 'DelayedArray', 'DelayedMatrixStats', 'DMRcate', 'DNAcopy',
  'DOSE', 'DSS', 'edgeR', 'enrichplot', 'ensembldb', 'fgsea', 'genefilter',
  'geneLenDataBase', 'GenomicAlignments', 'GenomicFeatures', 'GEOquery',
  'ggfun', 'ggtree', 'globaltest', 'GO.db', 'GOSemSim', 'goseq', 'graph',
  'graphlayouts', 'HDF5Array', 'Illumina450ProbeVariants.db', 'impute',
  'limma', 'lumi', 'marray', 'methylumi', 'minfi

In [19]:
library(OrganismDbi)
library(GenomicFeatures)
library(GOfuncR)
library(dplyr)
library(Homo.sapiens)

Loading required package: GenomicFeatures

Loading required package: GO.db



Loading required package: org.Hs.eg.db



Loading required package: TxDb.Hsapiens.UCSC.hg19.knownGene



In [55]:
Input_Genes <- read.delim2("/disk0/sm/bulk/test/output/gene_list.txt",header = T)[, c('Gene.name','logFC')]

In [32]:
## hyper

Go_Enrich_Out<- go_enrich(Input_Genes)
Results<-Go_Enrich_Out$results

Over_Representation<-Results[Results$raw_p_overrep<=0.05,]
Under_Representation<- Results[Results$raw_p_underrep<=0.05,]

Genes<- Go_Enrich_Out$genes
Candidate_Gene <-Genes[Genes$Is_candidate==1,]

Gene_all_GO<-get_anno_categories(Candidate_Gene$Gene_Ids)

Out_Over_Representation<-merge(Over_Representation,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)
Out_Under_Representation<-merge(Under_Representation,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)

Results_Over_Representation<- Out_Over_Representation %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit
Results_Under_Representation<- Out_Under_Representation %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit

Results_Over_Representation[ ,c(9,10)] <- NULL 
Results_Under_Representation[ ,c(9,10)] <- NULL

write.table(Results_Over_Representation,"/disk0/sm/bulk/test/output/korv1_Over_Representation",quote = FALSE, row.names = F,sep = "\t")
write.table(Results_Under_Representation,"/disk0/sm/bulk/test/output/korv1_Under_Representation",quote = FALSE, row.names = F,sep = "\t")

Checking arguments...



ERROR: Error in go_enrich(Input_Genes): Please provide a data frame with 2 columns [gene, 0/1] as input for hypergeometric test.


In [76]:
# wilcoxon <- read.delim2("/disk0/sm/PMDA/annotation/data/GO/data/korv1_gene_wilcoxon.txt",header = T)
wilcoxon <- read.delim2("/disk0/sm/bulk/test/output/gene_list.txt",header = T)[, c('Gene.name','logFC')]
wilcoxon$score <- exp(wilcoxon$logFC)
# wilcoxon %>% mutate(exp(logFC)) %>% head
colnames(wilcoxon) <- c('gene', 'score')
head(wilcoxon)

ERROR: Error in exp(wilcoxon$logFC): non-numeric argument to mathematical function


In [67]:
## wilcoxon

Go_Enrich_wilcoxon <- go_enrich(wilcoxon, test='wilcoxon')
Wilcoxon <- Go_Enrich_wilcoxon$results

Over_wilcoxon <-Wilcoxon[Wilcoxon$raw_p_high_rank <= 0.05,]
Under_wilcoxon <- Wilcoxon[Wilcoxon$raw_p_low_rank <= 0.05,]

Genes <- Go_Enrich_Out$genes

Gene_all_GO <- get_anno_categories(wilcoxon$SYMBOL)

Out_Over_wilcoxon <- merge(Over_wilcoxon,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)
Out_Under_wilcoxon <- merge(Under_wilcoxon,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)

Results_Over_wilcoxon <- Out_Over_wilcoxon %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit
Results_Under_wilcoxon <- Out_Under_wilcoxon %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit

Results_Over_wilcoxon[ ,c(9,10)] <- NULL 
Results_Under_wilcoxon[ ,c(9,10)] <- NULL

write.table(Results_Over_wilcoxon,"/disk0/sm/bulk/test/output/Over_wilcoxon",quote = FALSE, row.names = F,sep = "\t")
write.table(Results_Under_wilcoxon,"/disk0/sm/bulk/test/output/Under_wilcoxon",quote = FALSE, row.names = F,sep = "\t")

Checking arguments...



ERROR: Error in go_enrich(wilcoxon, test = "wilcoxon"): Please provide a data frame with 2 columns [gene, score] as input for wilcoxon rank-sum test.


In [ ]:
## KORV2

In [4]:
Input_Genes <- read.delim2("/disk0/sm/PMDA/annotation/data/GO/data/korv2_gene.txt",header = T)


In [5]:
## hyper

Go_Enrich_Out<- go_enrich(Input_Genes)
Results<-Go_Enrich_Out$results

Over_Representation<-Results[Results$raw_p_overrep<=0.05,]
Under_Representation<- Results[Results$raw_p_underrep<=0.05,]

Genes<- Go_Enrich_Out$genes
Candidate_Gene <-Genes[Genes$Is_candidate==1,]

Gene_all_GO<-get_anno_categories(Candidate_Gene$Gene_Ids)

Out_Over_Representation<-merge(Over_Representation,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)
Out_Under_Representation<-merge(Under_Representation,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)

Results_Over_Representation<- Out_Over_Representation %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit
Results_Under_Representation<- Out_Under_Representation %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit

Results_Over_Representation[ ,c(9,10)] <- NULL 
Results_Under_Representation[ ,c(9,10)] <- NULL

write.table(Results_Over_Representation,"data/korv2_Over_Representation",quote = FALSE, row.names = F,sep = "\t")
write.table(Results_Under_Representation,"data/korv2_Under_Representation",quote = FALSE, row.names = F,sep = "\t")

Checking arguments...



Write temporary files...

Find associated categories using database 'Homo.sapiens'...

Remove annotated categories not present in GO-graph...

Found root_nodes: biological_process, cellular_component, molecular_function

Remove invalid genes...

Warning message in go_enrich(Input_Genes):
“No GO-annotation for genes: POLR1HASP, WASF5P, Y_RNA, MIR3681HG, PKN2-AS1, TSBP1-AS1, LINC02694, HCG20, LINC01505, LINC00299, CCDC26, CASC15, PCAT1, DIRC3-AS1, LINC00243, LINC02571, LINC01149, HCP5, MIR4435-2HG, LINC02055, PSORS1C1, LINC01320, LINC01358, SATB1-AS1, MIR99AHG, LINC00598, MIR100HG, EPIC1, PPP1R3B-DT, LINC02240, LINC02934, DPYD-AS1, CYYR1-AS1, CCSER1, COX10-DT, MIR3667HG, HLA-DRB9, FHAD1, PVT1, HCG18, WDR64, SLC44A3-AS1, LMCD1-AS1, SGO1-AS1, SPRY4-AS1, LINC00540, LINC01036, ZDHHC20P2, NALCN-AS1, CFAP20DC-DT, STX18-AS1, LINC01500, DRAIC, VLDLR-AS1, CCDC162P, LINC00923, MIR6891, USP8P1, PWRN1, TRD-AS1, CRAT37, FAM107B, SOX9-AS1, HCG17, SLC8A1-AS1, LINC01060, LINC01250, FRMD6-AS2, MICB-DT, L

Read 47205 terms.
Found 28642 nodes.
Graph created.
Reading nodes_per_gene file... 
Found 18613 usable entrys in /tmp/RtmpKIgck6/file3e57d4d7b9fd8_GO:0008150 with 16038 GOs
Creating 1000 randomsets with size 10030
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
under-representation			over-representation
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
760	315	52	8	0	752	337	66	2	0	
716	295	56	9	0	770	354	42	3	0	
934	401	84	7	1	536	207	14	0	0	
1118	559	96	5	0	622	261	43	3	0	
990	413	71	8	0	597	240	34	0	0	
728	274	38	2	0	704	282	39	3	0	
804	341	51	2	0	746	339	50	6	1	
728	295	45	1	0	758	292	50	2	0	
791	328	40	2	0	680	279	45	1	0	
796	334	52	0	0	769	345	49	2	0	
795	334	37	0	0	624	255	28	1	0	
771	314	47	3	0	686	302	57	12	0	
762	322	68	9	1	710	290	50	4	2	
686	272	42	3	0	715	298	42	1	0	
861	353	60	3	0	714	324	53	3	0	
637	274	32	0	0	1037	490	110	9	3	
626	242	28	0	0	742	277	35	2	1	
719	300	48	4	0	970	481	115	12	0	
920	405	59	5



Processing root node: cellular_component...


Run Func...




Read 47205 terms.
Found 4176 nodes.
Graph created.
Reading nodes_per_gene file... 
Found 19518 usable entrys in /tmp/RtmpKIgck6/file3e57d4d7b9fd8_GO:0005575 with 1971 GOs
Creating 1000 randomsets with size 10455
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
under-representation			over-representation
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
97	55	6	1	0	77	27	3	0	0	
72	32	1	0	0	114	36	4	0	0	
117	60	15	0	0	68	22	1	0	0	
75	40	7	0	0	94	47	8	1	0	
89	34	4	0	0	74	34	3	0	0	
108	53	2	1	0	65	27	2	0	0	
106	66	14	1	0	85	42	10	0	0	
73	34	7	0	0	103	59	11	0	0	
105	46	5	1	1	84	33	6	0	0	
64	31	6	0	0	130	49	7	0	0	
89	38	3	0	0	75	26	2	0	0	
52	27	5	0	0	71	23	1	0	0	
101	47	5	0	0	66	34	5	0	0	
110	56	7	1	0	70	31	2	0	0	
85	45	2	0	0	90	36	8	2	0	
82	43	4	0	0	89	36	10	1	1	
79	41	3	0	0	94	42	8	1	1	
100	50	5	0	0	72	25	3	0	0	
115	63	5	1	0	74	30	2	0	0	
79	34	4	0	0	84	28	10	0	0	
74	26	6	2	1	108	43	1	0	0	
105	54	4	1	0	81	22	4	1	0	
83	42	3	1



Processing root node: molecular_function...


Run Func...




Read 47205 terms.
Found 11169 nodes.
Graph created.
Reading nodes_per_gene file... 
Found 18364 usable entrys in /tmp/RtmpKIgck6/file3e57d4d7b9fd8_GO:0003674 with 4908 GOs
Creating 1000 randomsets with size 10294
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
under-representation			over-representation
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
188	82	16	0	0	99	42	7	0	0	
193	79	15	1	0	166	70	12	1	0	
179	69	8	0	0	151	61	21	1	0	
138	52	3	2	0	155	58	11	1	0	
165	69	3	0	0	161	55	1	0	0	
180	65	11	0	0	146	57	14	1	1	
135	49	7	0	0	170	70	11	2	0	
205	92	9	2	0	157	67	13	0	0	
130	53	2	0	0	181	64	8	0	0	
209	83	12	0	0	201	94	22	0	0	
218	81	11	0	0	139	53	7	1	0	
183	85	14	5	0	111	34	4	0	0	
180	74	10	0	0	159	52	6	0	0	
166	60	9	0	0	167	72	10	1	0	
146	49	9	2	0	142	59	4	0	0	
189	87	16	1	0	143	56	12	1	0	
188	80	14	1	0	153	61	15	3	1	
194	76	10	0	0	130	46	8	2	0	
167	62	10	0	0	138	47	6	0	0	
162	58	10	1	0	152	60	12	3	0	
164	67	12	2	0	1


Done.

Find associated categories using database 'Homo.sapiens'...

Remove annotated categories not present in GO-graph...



In [6]:
wilcoxon <- read.delim2("/disk0/sm/PMDA/annotation/data/GO/data/korv2_gene_wilcoxon.txt",header = T)

In [7]:
## wilcoxon

# Go_Enrich_wilcoxon <- go_enrich(wilcoxon, test='wilcoxon')
# Wilcoxon <- Go_Enrich_wilcoxon$results

# Over_wilcoxon <-Wilcoxon[Wilcoxon$raw_p_high_rank <= 0.05,]
# Under_wilcoxon <- Wilcoxon[Wilcoxon$raw_p_low_rank <= 0.05,]

# Genes <- Go_Enrich_Out$genes

# Gene_all_GO <- get_anno_categories(wilcoxon$SYMBOL)

# Out_Over_wilcoxon <- merge(Over_wilcoxon,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)
# Out_Under_wilcoxon <- merge(Under_wilcoxon,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)

# Results_Over_wilcoxon <- Out_Over_wilcoxon %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit
# Results_Under_wilcoxon <- Out_Under_wilcoxon %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit

# Results_Over_wilcoxon[ ,c(9,10)] <- NULL 
# Results_Under_wilcoxon[ ,c(9,10)] <- NULL

# write.table(Results_Over_wilcoxon,"data/korv2_Over_wilcoxon",quote = FALSE, row.names = F,sep = "\t")
# write.table(Results_Under_wilcoxon,"data/korv2_Under_wilcoxon",quote = FALSE, row.names = F,sep = "\t")

Checking arguments...

Write temporary files...

Find associated categories using database 'Homo.sapiens'...

Remove annotated categories not present in GO-graph...

Found root_nodes: biological_process, cellular_component, molecular_function

Remove invalid genes...

Warning message in go_enrich(wilcoxon, test = "wilcoxon"):
“No GO-annotation for genes: POLR1HASP, WASF5P, Y_RNA, MIR3681HG, PKN2-AS1, TSBP1-AS1, LINC02694, HCG20, LINC01505, LINC00299, CCDC26, CASC15, PCAT1, DIRC3-AS1, LINC00243, LINC02571, LINC01149, HCP5, MIR4435-2HG, LINC02055, PSORS1C1, LINC01320, LINC01358, SATB1-AS1, MIR99AHG, LINC00598, MIR100HG, EPIC1, PPP1R3B-DT, LINC02240, LINC02934, DPYD-AS1, CYYR1-AS1, CCSER1, COX10-DT, MIR3667HG, HLA-DRB9, FHAD1, PVT1, HCG18, WDR64, SLC44A3-AS1, LMCD1-AS1, SGO1-AS1, SPRY4-AS1, LINC00540, LINC01036, ZDHHC20P2, NALCN-AS1, CFAP20DC-DT, STX18-AS1, LINC01500, DRAIC, VLDLR-AS1, CCDC162P, LINC00923, MIR6891, USP8P1, PWRN1, TRD-AS1, CRAT37, FAM107B, SOX9-AS1, HCG17, SLC8A1-AS1, LINC

Read 47205 terms.
Found 28642 nodes.
Graph created.
Data and annotation file parsed.
Number of randomsets: 1000.
Computing randomsets...1000.
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
low ranks				high ranks
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
1382	632	105	15	1	1632	727	79	4	0	
1015	403	54	3	0	1629	754	96	7	1	
1335	544	88	3	0	1656	759	112	14	2	
1173	494	59	2	0	1732	735	102	13	2	
1063	410	36	0	0	1575	655	74	2	0	
1138	499	75	5	0	1800	776	127	19	3	
980	359	48	3	0	1804	774	94	3	0	
982	407	48	1	0	1856	767	117	4	0	
1187	540	66	3	0	1822	797	117	3	1	
1160	447	55	4	0	1866	884	127	8	0	
1081	490	64	1	0	1977	938	141	8	0	
1155	476	64	4	0	1827	886	162	19	0	
1275	535	71	2	0	1597	635	74	8	0	
1357	630	113	14	1	1621	679	110	13	0	
1286	601	84	7	0	1517	590	70	7	0	
1281	537	97	5	0	1417	622	65	4	0	
1329	533	52	4	0	1488	600	94	3	0	
1037	442	56	6	0	1774	830	111	6	0	
1104	460	60	2	0	1948	901	134	7	0	
1422	623	99	7	0	1419	6



Processing root node: cellular_component...


Run Func...




Read 47205 terms.
Found 4176 nodes.
Graph created.
Data and annotation file parsed.
Number of randomsets: 1000.
Computing randomsets...1000.
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
low ranks				high ranks
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
141	40	4	0	0	198	82	6	3	0	
158	67	9	1	0	189	63	6	0	0	
146	61	3	0	0	232	92	18	0	0	
148	52	14	0	0	241	111	20	2	0	
159	62	3	0	0	197	95	9	0	0	
124	33	3	0	0	261	117	21	1	0	
155	61	4	1	0	209	78	14	0	0	
171	68	8	0	0	224	98	25	2	1	
156	61	9	0	0	215	89	16	1	0	
188	85	16	0	0	202	88	17	1	1	
135	64	7	0	0	184	88	10	0	0	
110	46	1	0	0	195	82	12	0	0	
143	69	18	3	0	203	81	5	0	0	
174	76	8	1	0	188	80	5	0	0	
150	59	4	0	0	174	73	8	0	0	
156	81	19	3	0	226	87	17	0	0	
102	35	7	1	0	267	109	21	0	0	
157	61	14	0	0	198	91	14	0	0	
179	55	12	3	0	199	82	8	0	0	
113	52	5	0	0	221	112	7	0	0	
173	63	6	0	0	230	108	9	3	0	
163	68	10	1	0	161	64	11	1	0	
222	109	19	3	0	190	87	5	0	0	
126	58	3	0	0	236	103



Processing root node: molecular_function...


Run Func...




Read 47205 terms.
Found 11169 nodes.
Graph created.
Data and annotation file parsed.
Number of randomsets: 1000.
Computing randomsets...1000.
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
low ranks				high ranks
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
327	125	25	5	0	586	234	32	1	0	
306	103	16	4	0	523	216	29	2	1	
310	90	10	0	0	506	237	35	2	0	
296	105	9	0	0	541	197	18	0	0	
303	96	9	1	0	601	230	20	1	0	
302	92	11	0	0	463	195	18	0	0	
327	113	8	0	0	540	224	21	1	0	
352	117	17	3	0	526	216	36	2	0	
334	114	10	0	0	520	238	23	1	0	
295	84	7	0	0	509	227	25	1	0	
288	110	11	0	0	614	245	27	3	1	
304	95	12	1	0	490	200	35	1	0	
317	102	13	1	0	521	209	17	0	0	
291	88	12	2	0	479	178	16	0	0	
273	99	12	0	0	502	212	26	1	0	
308	111	10	1	0	521	221	15	3	1	
334	125	11	2	0	503	203	21	2	2	
280	100	11	0	0	526	220	17	0	0	
315	107	12	2	0	459	197	22	3	0	
287	97	8	0	0	525	208	17	0	0	
240	73	7	0	0	566	271	26	2	0	
323	127	16	2	0	515	226	25	3	0	



Done.

Find associated categories using database 'Homo.sapiens'...

Remove annotated categories not present in GO-graph...



In [ ]:
## KORV1 KORV2 각각에만 있는 gene중 KORV2꺼만 후보진으로 하여 분석 진행

In [7]:
Input_Genes <- read.delim2("/disk0/sm/PMDA/annotation/data/GO/data/gene.txt",header = T)

In [8]:
## hyper

Go_Enrich_Out<- go_enrich(Input_Genes)
Results<-Go_Enrich_Out$results

Over_Representation<-Results[Results$raw_p_overrep<=0.05,]
Under_Representation<- Results[Results$raw_p_underrep<=0.05,]

Genes<- Go_Enrich_Out$genes
Candidate_Gene <-Genes[Genes$Is_candidate==1,]

Gene_all_GO<-get_anno_categories(Candidate_Gene$Gene_Ids)

Out_Over_Representation<-merge(Over_Representation,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)
Out_Under_Representation<-merge(Under_Representation,Gene_all_GO, by.x = "node_id",by.y = "go_id", all.x = TRUE, all.y = FALSE)

Results_Over_Representation<- Out_Over_Representation %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit
Results_Under_Representation<- Out_Under_Representation %>% group_by(node_id) %>% mutate(gene= paste(gene, collapse=",")) %>% unique %>% na.omit

Results_Over_Representation[ ,c(9,10)] <- NULL 
Results_Under_Representation[ ,c(9,10)] <- NULL

write.table(Results_Over_Representation,"data/Over_Representation",quote = FALSE, row.names = F,sep = "\t")
write.table(Results_Under_Representation,"data/Under_Representation",quote = FALSE, row.names = F,sep = "\t")

Checking arguments...



Write temporary files...

Find associated categories using database 'Homo.sapiens'...

Remove annotated categories not present in GO-graph...

Found root_nodes: biological_process, cellular_component, molecular_function

Remove invalid genes...

Warning message in go_enrich(Input_Genes):
“No GO-annotation for genes: NDUFA6-DT, CYP2F2P, LINC00454, KRT18P51, WDR87, RPL31P13, NCKAP5-AS2, PCAT19, ACSL3P1, IGLV3-29, ZNF470-DT, CEP164P1, NUTM2B-AS1, LINC02642, CYCSP53, IKBKE-AS1, LINC01353, LINC00452, POC1B-AS1, LINC03081, PINK1-AS, XRCC6P3, STARP1, TP73-AS1, CFAP57, UPK1A-AS1, LINC01766, LMO7DN, RNY4P27, PRDM16-DT, CENPBD2P, RNU6-239P, RIIAD1, OR6K4P, OR10AE1P, LINC01342, NUCB1-AS1, TEX21P, ACTG1P25, LINC02319, HIF1A-AS1, PNMA8A, USP3-AS1, RN7SL466P, PFN1P1, ALKBH3-AS1, KPNB1-DT, CCDC34, RPL26P31, LINC02553, SBF2-AS1, KRT73-AS1, RNU4-17P, CYP2C23P, EBLN1, ZNF271P, RPSAP67, DNAJB6P4, BCL10-AS1, FAM238C, MIR4679-2, LINC02179, LINC02178, LINC01900, RNU6-1244P, RPL26P28, RNU6-391P, NCRNA00250, 

Read 47205 terms.
Found 28642 nodes.
Graph created.
Reading nodes_per_gene file... 
Found 13053 usable entrys in /tmp/RtmpaEdi0u/filec9b27bd3fb4c_GO:0008150 with 15320 GOs
Creating 1000 randomsets with size 1856
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
under-representation			over-representation
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
366	152	21	0	0	579	237	40	3	0	
336	134	13	2	0	723	328	66	6	2	
412	199	38	3	0	637	285	32	1	0	
400	198	30	2	0	613	227	29	4	0	
574	272	56	8	0	519	208	35	5	1	
268	111	11	1	0	782	336	40	3	0	
355	142	27	0	0	606	248	32	6	1	
366	164	32	2	0	754	296	38	3	0	
324	156	22	1	0	728	307	34	5	0	
407	194	28	1	1	710	309	63	6	0	
420	202	37	2	0	588	259	37	2	1	
288	127	18	1	0	778	340	57	0	0	
498	202	30	1	0	546	220	41	3	0	
350	157	36	2	0	703	286	48	6	0	
350	150	24	0	0	597	245	40	1	0	
468	241	76	20	5	572	214	27	1	0	
216	74	6	0	0	951	433	91	10	0	
276	111	19	1	0	862	404	72	8	0	
336	128	12	0	0	549	2



Processing root node: cellular_component...


Run Func...




Read 47205 terms.
Found 4176 nodes.
Graph created.
Reading nodes_per_gene file... 
Found 13652 usable entrys in /tmp/RtmpaEdi0u/filec9b27bd3fb4c_GO:0005575 with 1923 GOs
Creating 1000 randomsets with size 1932
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
under-representation			over-representation
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
42	14	2	0	0	74	27	5	0	0	
41	11	1	0	0	118	45	6	0	0	
48	15	1	0	0	98	35	4	0	0	
51	23	1	0	0	98	46	9	0	0	
36	7	1	0	0	97	42	13	0	0	
29	14	4	0	0	139	74	26	3	0	
74	32	4	1	0	70	28	9	0	0	
60	25	0	0	0	95	47	8	0	0	
57	24	7	0	0	75	29	3	0	0	
27	11	4	2	0	126	53	3	0	0	
45	24	13	1	0	72	26	1	0	0	
54	27	4	0	0	99	34	15	4	0	
60	33	4	0	0	59	21	4	1	0	
41	11	3	0	0	123	48	11	0	0	
40	15	0	0	0	93	45	12	0	0	
52	21	1	0	0	105	36	11	2	0	
36	14	0	0	0	105	57	9	0	0	
51	23	1	1	0	88	40	8	0	0	
62	32	3	1	0	81	39	6	0	0	
21	5	1	0	0	94	30	9	1	1	
37	19	3	0	0	97	29	3	0	0	
44	21	2	0	0	110	43	9	2	0	
19	12	5	1	0	97	27	



Processing root node: molecular_function...


Run Func...




Read 47205 terms.
Found 11169 nodes.
Graph created.
Reading nodes_per_gene file... 
Found 13443 usable entrys in /tmp/RtmpaEdi0u/filec9b27bd3fb4c_GO:0003674 with 4577 GOs
Creating 1000 randomsets with size 1923
Finished

Evaluating randomsets: 
No. of significant ontology nodes for
under-representation			over-representation
of candidate genes at p-value thresholds
0.1	0.05	0.01	0.001	0.0001	0.1	0.05	0.01	0.001	0.0001
52	9	1	0	0	150	72	6	0	0	
81	28	2	0	0	129	62	10	5	0	
59	25	4	2	0	211	99	12	2	0	
38	12	1	0	0	151	69	7	1	0	
50	19	3	1	0	138	75	9	3	0	
56	9	0	0	0	149	87	9	1	0	
50	20	2	0	0	182	91	10	1	0	
80	35	5	0	0	114	56	6	2	2	
50	19	6	0	0	154	67	13	0	0	
72	27	4	0	0	135	70	9	1	0	
56	19	3	0	0	130	53	3	0	0	
55	18	2	0	0	106	51	4	0	0	
28	6	0	0	0	165	70	7	1	0	
63	26	3	0	0	155	80	15	0	0	
73	31	4	1	0	158	87	18	1	0	
62	21	2	0	0	171	82	19	1	0	
36	13	0	0	0	163	77	12	1	0	
96	30	11	0	0	140	69	13	2	0	
43	13	0	0	0	166	89	10	0	0	
59	21	5	0	0	125	58	8	2	0	
86	34	5	0	0	156	74	12	0	0	
83	29	3	0	0	134	60	9	0	0


Done.

Find associated categories using database 'Homo.sapiens'...

Remove annotated categories not present in GO-graph...

